## Dataset Ideas
**https://data.wprdc.org/dataset/arrest-data** Maybe group by types of crime and weight differently?

**https://data.wprdc.org/dataset/pgh/resource/b7156251-6036-4b68-ad2a-95566c84343e** Neighborhood Population Data

https://data.wprdc.org/dataset/playgrounds

https://data.wprdc.org/dataset/smart-trash-containers

In [2]:
import pandas as pd
import numpy as np

In [3]:
arrest_data = pd.read_csv("arrest-data.csv")
census_data = pd.read_excel("census-data.xlsx")

C:\Users\pnmar\anaconda3\envs\cmpinf0010\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
arrest_data.sample(3)

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y
23753,2004684,18165571,23.0,M,B,2018-08-26T17:09:00,"2900 Block N Carey WY Pittsburgh, PA 15203",3925 Receiving Stolen Property. / 5121 Escape....,"2800 Block Jane ST Pittsburgh, PA 15203",South Side Flats,3,1609.0,3.0,3.0,-79.967236,40.424807
35495,2019728,19201526,29.0,M,B,2019-10-03T20:58:00,"Tabor ST & Harrison WY Pittsburgh, PA 15204",13(a)(31) Marijuana: Possession Small Amount,"Tabor ST & Harrison WY Pittsburgh, PA 15204",Esplen,6,2017.0,2.0,5.0,-80.055227,40.461601
9756,1986849,17106137,42.0,M,B,2017-06-22T08:00:00,"20 Block E Meyers ST Pittsburgh, PA 15210",6106 Firearms not to be Carried without a Lice...,Saw Mill Run Blvd RAMP & West End Cir RAMP Pit...,Elliott,6,2020.0,NaN,NaN,0.000000,0.000000


In [4]:
# CRIME TYPES AND WEIGHTS
# Theft 4
# Burglary 4
# Simple Assault 2
# Aggravated Assault 4
# Homicide 10
# Robbery 4
# Kidnapping 8

# Idea: Get total offenses by neighborhood
#       Get number of different types of crimes by neighborhood
#       Multiply the crime types by (weight - 1) (so we can add their values to the total offenses by neighborhood)
#       Add the crime types by neighborhood value to total offenses
#       Divide this number by the population * some constant (maybe weighted crime per 10k or something)
#       Graph total offenses by neighborhood, different crimes by neighborhood, crimes per capita, weighted crimes per capita

# Creates a series for each crime in 'crimeTypes', containing the number of instances of that crime
# Each crime series is added to the dictionary 'crimeList'
def addCrimes(crimeTypes, crimeList):
    otherMask = offenses.str.contains("ABCDEFGHIJKLMNOP") # Should be false for everything
    
    for crime in crimeTypes:
        mask = offenses.str.contains(crime, na=False)
        a = arrest_data[mask].groupby("INCIDENTNEIGHBORHOOD")["OFFENSES"].count()
        crimeList[crime] = a
        otherMask = mask | otherMask # Sets any rows we used to true
    
    # All rows we DIDN'T use are added as "Other" (note the ~)
    a = arrest_data[~otherMask].groupby("INCIDENTNEIGHBORHOOD")["OFFENSES"].count()
    crimeList["Other"] = a
    

offenses = arrest_data["OFFENSES"]

# Group crimes by neighborhood into a dictionary
crimeList = {}
addCrimes(["Theft", "Burglary", "Simple Assault", "Aggravated Assault", "Homicide", "Robbery", "Kidnapping"], crimeList)


# Put all crime types into one DataFrame
crimeInstances = pd.DataFrame(crimeList)

# merge crime types with population
cd = census_data.set_index("Neighborhood")["Pop. 2010"]
crimeInstances = crimeInstances.merge(cd, how='outer', left_index=True, right_index=True)
crimeInstances.fillna(0)

,Theft,Burglary,Simple Assault,Aggravated Assault,Homicide,Robbery,Kidnapping,Other,Pop. 2010
Allegheny Center,36.0,7.0,70.0,62.0,1.0,23.0,4.0,690.0,933.0
Allegheny West,10.0,6.0,11.0,3.0,0.0,0.0,0.0,63.0,462.0
Allentown,36.0,16.0,169.0,37.0,1.0,16.0,1.0,442.0,2500.0
Arlington,18.0,7.0,65.0,25.0,0.0,10.0,0.0,100.0,1869.0
Arlington Heights,3.0,0.0,36.0,29.0,0.0,7.0,0.0,43.0,244.0
...,...,...,...,...,...,...,...,...,...
Upper Lawrenceville,19.0,11.0,43.0,6.0,0.0,4.0,1.0,77.0,2669.0
West End,8.0,4.0,21.0,7.0,2.0,6.0,4.0,156.0,254.0
West Oakland,24.0,11.0,59.0,17.0,2.0,9.0,2.0,126.0,2604.0
Westwood,33.0,3.0,50.0,11.0,0.0,10.0,0.0,68.0,3066.0


I created the new dataset with the split-apart crimes and population. From this alone, there is no way to analyze the data yet, but it's a great starting point and it's almost able to be analyzed properly. My group members did the rest of the work, and ultimately it appears that ______ was the best neighborhood because it had the lowest weighted crime/population value.